In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import mbuild as mb
import foyer
from foyer import Forcefield

import mbuild.formats.charmm_writer as mf_charmm

import mosdef_cp2k_writer


import parmed as pmd
from constrainmol import ConstrainedMolecule
import unyt as u

from mosdef_cp2kmcpy.mc import MC
import setter
warnings.filterwarnings("ignore", category=DeprecationWarning)
 

In [3]:
class O2(mb.Compound): # this class builds an oxygen molecule with a bond-length given in the oxygen2 x coor (nm)
    def __init__(self):
        super(O2, self).__init__()

        oxygen1= mb.Particle(pos=[0.0, 0.0, 0.0], name='O', element = "O")
        oxygen2= mb.Particle(pos=[0.15, 0.0, 0.0], name='O', element = "O")
        self.add([oxygen2,oxygen1])
        self.add_bond((oxygen2,oxygen1))

molecule=O2();

In [4]:
for particle in molecule.particles():
    print(particle.name)
    print(particle.element.symbol)

O
O
O
O


In [5]:

hydrogen_res_name = 'H2'
FF_file_hydrogen = 'h2_ff.xml'
hydrogen_FF = Forcefield(forcefield_files=FF_file_hydrogen)
hydrogen_mb = mb.load('[HH]', smiles=True)

hydrogen_mb.name = hydrogen_res_name

hydrogen_typed=hydrogen_FF.apply(hydrogen_mb)



constrain_mol = ConstrainedMolecule(hydrogen_typed)
constrain_mol.solve()
hydrogen_optimized_mb=mb.clone(hydrogen_mb)
hydrogen_optimized_mb.xyz=constrain_mol.xyz/10
print(hydrogen_optimized_mb.xyz)

/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/foyer/forcefield.py:448: UserWarning: No force field version number found in force field XML file.
  warnings.warn(
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/foyer/forcefield.py:460: UserWarning: No force field name found in force field XML file.
  warnings.warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:687: UserWarning: No unitcell detected for pybel.Molecule [H][H]	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


[[ 0.09864176 -0.00149139  0.00766917]
 [ 0.17274176 -0.00149139  0.00766917]]


In [6]:
hydrogen_optimized_mb.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
for particle in hydrogen_optimized_mb.particles():
    particle.name = "HH"
    particle.element = "H"

In [8]:
for particle in hydrogen_optimized_mb.particles():
    
    print(particle.name)
    print(particle.element.symbol)

HH
H
HH
H


In [9]:

nitrogen_res_name = 'N2'
FF_file_nitrogen = 'dinitrogen_ff.xml'
nitrogen_FF = Forcefield(forcefield_files=FF_file_nitrogen)
nitrogen_mb = mb.load('N#N', smiles=True)

nitrogen_mb.name = nitrogen_res_name

nitrogen_typed=nitrogen_FF.apply(nitrogen_mb)



constrain_mol = ConstrainedMolecule(nitrogen_typed)
constrain_mol.solve()
nitrogen_optimized_mb=mb.clone(nitrogen_mb)
nitrogen_optimized_mb.xyz=constrain_mol.xyz/10
print(nitrogen_optimized_mb.xyz)

/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:687: UserWarning: No unitcell detected for pybel.Molecule N#N	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


[[ 0.10526547  0.00909225 -0.00414265]
 [ 0.21736547  0.00909225 -0.00414265]]


In [10]:
nitrogen_optimized_mb.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [11]:
for particle in nitrogen_optimized_mb.particles():
    particle.name = "NN"
    particle.element = "N"
    print(particle.name)
    print(particle.element.symbol)

NN
N
NN
N


## Creating .psf (Topology) and .inp (CHARMM potential file) using mosdef_charmm_writer


In [19]:
#random box to be used in mosdef_charmm_writer
hydrogen_box_bias  = mb.fill_box(compound=[hydrogen_optimized_mb ],
                             n_compounds=[1] ,
                            box=[1.0, 1.0, 1.0] )

nitrogen_box_bias  = mb.fill_box(compound=[nitrogen_optimized_mb ],
                             n_compounds=[1] ,
                            box=[1.0, 1.0, 1.0] )

/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.0986,-0.0015, 0.0077), 0 bonds, id: 140346456032448>" is element: "H"
  warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<NN pos=( 0.1053, 0.0091,-0.0041), 0 bonds, id: 140347864938528>" is element: "N"
  warn(


In [20]:
mixed_box  = mb.fill_box(compound=[hydrogen_optimized_mb, nitrogen_box_bias ],
                             n_compounds=[1, 1] ,
                            box=[1.0, 1.0, 1.0] )

/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<NN pos=( 0.0001, 0.0383, 0.7949), 0 bonds, id: 140347865224816>" is element: "N"
  warn(


In [21]:
FF_Dict = {hydrogen_optimized_mb.name:FF_file_hydrogen, nitrogen_optimized_mb.name:FF_file_nitrogen }

residues_List = [hydrogen_optimized_mb.name, nitrogen_optimized_mb.name ]




#hydrogen_box_bias.save("hydrogen_bias_coord.xyz",overwrite=True)
#nitrogen_box_bias.save("nitrogen_bias_coord.xyz",overwrite=True)

print('Running: GOMC FF file, and the psf and pdb files for the biasing potential file')
mf_charmm.charmm_psf_psb_FF(mixed_box ,
                            'overall_bias',
                            FF_filename ="overall_charmm_bias" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            bead_to_atom_name_dict = {},
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )

! rm *.pdb *.xyz

Running: GOMC FF file, and the psf and pdb files for the biasing potential file
write_gomcdata: forcefield_selection = {'H2': 'h2_ff.xml', 'N2': 'dinitrogen_ff.xml'}, residues = ['H2', 'N2']
******************************

GOMC FF writing each residues FF as a group for structure_0
forcefield type from compound = {'H2': 'h2_ff.xml', 'N2': 'dinitrogen_ff.xml'}
coulomb14scale from compound = {'H2': 0.0, 'N2': 0.0}
lj14scale from compound = {'H2': 0.0, 'N2': 0.0}
unique_types = ['H_H2', 'N_N2']
No urey bradley terms detected, will use angle_style harmonic
******************************

writing the GOMC force field file 
NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
forcefield_dict = {0: 'H_H2', 1: 'N_N2'}
******************************

write_charmm_psf file is running
write_charmm_psf: forcefield_selection = {'H2': 'h2_ff.xml', 'N2': 'dinitrogen_ff.xml'}, residues = ['H2', 'N2']
******************************

No urey bradley terms detected
******************

/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.0641, 0.1072, 0.8047), 0 bonds, id: 140347865274016>" is element: "H"
  warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<NN pos=( 0.2540, 0.0033, 0.5706), 0 bonds, id: 140347865275840>" is element: "N"
  warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.0641, 0.1072, 0.8047), 0 bonds, id: 140347864534560>" is element: "H"
  warn(
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<NN pos=( 0.2540, 0.0033, 0.5706), 0 b

In [22]:
FF_Dict = {hydrogen_optimized_mb.name:FF_file_hydrogen }

residues_List = [hydrogen_optimized_mb.name ]




#hydrogen_box_bias.save("hydrogen_bias_coord.xyz",overwrite=True)
#nitrogen_box_bias.save("nitrogen_bias_coord.xyz",overwrite=True)

print('Running: GOMC FF file, and the psf and pdb files for the biasing potential file')
mf_charmm.charmm_psf_psb_FF(hydrogen_box_bias ,
                            'hydrogen_bias',
                            FF_filename ="hydrogen_charmm_bias" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            bead_to_atom_name_dict = {},
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )

! rm *.pdb *.xyz

Running: GOMC FF file, and the psf and pdb files for the biasing potential file
write_gomcdata: forcefield_selection = {'H2': 'h2_ff.xml'}, residues = ['H2']
******************************

GOMC FF writing each residues FF as a group for structure_0
forcefield type from compound = {'H2': 'h2_ff.xml'}
coulomb14scale from compound = {'H2': 0.0}
lj14scale from compound = {'H2': 0.0}
unique_types = ['H_H2']
No urey bradley terms detected, will use angle_style harmonic
******************************

writing the GOMC force field file 
NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
forcefield_dict = {0: 'H_H2'}
******************************

write_charmm_psf file is running
write_charmm_psf: forcefield_selection = {'H2': 'h2_ff.xml'}, residues = ['H2']
******************************

No urey bradley terms detected
******************************

write_charmm_pdb file is running
write_charmm_pdb: residues == ['H2']
fix_residue = None
fix_residue_in_box = None
bead_

/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.0166, 0.0275, 0.7999), 0 bonds, id: 140347865244048>" is element: "H"
  warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.0166, 0.0275, 0.7999), 0 bonds, id: 140347863649488>" is element: "H"
  warn(
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


In [23]:
FF_Dict = { nitrogen_optimized_mb.name:FF_file_nitrogen }

residues_List = [nitrogen_optimized_mb.name ]




#hydrogen_box_bias.save("hydrogen_bias_coord.xyz",overwrite=True)
#nitrogen_box_bias.save("nitrogen_bias_coord.xyz",overwrite=True)

print('Running: GOMC FF file, and the psf and pdb files for the biasing potential file')
mf_charmm.charmm_psf_psb_FF(nitrogen_box_bias ,
                            'nitrogen_bias',
                            FF_filename ="nitrogen_charmm_bias" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            bead_to_atom_name_dict = {},
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )

! rm *.pdb *.xyz

Running: GOMC FF file, and the psf and pdb files for the biasing potential file
write_gomcdata: forcefield_selection = {'N2': 'dinitrogen_ff.xml'}, residues = ['N2']
******************************

GOMC FF writing each residues FF as a group for structure_0
forcefield type from compound = {'N2': 'dinitrogen_ff.xml'}
coulomb14scale from compound = {'N2': 0.0}
lj14scale from compound = {'N2': 0.0}
unique_types = ['N_N2']
No urey bradley terms detected, will use angle_style harmonic
******************************

writing the GOMC force field file 
NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
forcefield_dict = {0: 'N_N2'}
******************************

write_charmm_psf file is running
write_charmm_psf: forcefield_selection = {'N2': 'dinitrogen_ff.xml'}, residues = ['N2']
******************************

No urey bradley terms detected
******************************

write_charmm_pdb file is running
write_charmm_pdb: residues == ['N2']
fix_residue = None
fix_res

/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<NN pos=( 0.0001, 0.0383, 0.7949), 0 bonds, id: 140347863855504>" is element: "N"
  warn(
/home/rs/anaconda3/envs/cp2k_mc/lib/python3.8/site-packages/parmed/openmm/topsystem.py:236: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  warnings.warn('Adding what seems to be Urey-Bradley terms before ' # pragma: no cover


### Now we have the .psf (topology) file and .inp (charmm potential) file

## Creating CP2K MC simulation

In [12]:
molecule_list=[hydrogen_optimized_mb, nitrogen_optimized_mb]
box=mb.box.Box(lengths=[1,1,1])
#Comment why pressure is needed


In [13]:
q=MC(molecule_list=molecule_list,n_box=2,n_molecules_each_box=[[2,2],[2,2]], box_list=[box,box],cutoff=200,functional='PBE',
     basis_set={'NN':'DZVP-MOLOPT-GTH','HH':'DZVP-MOLOPT-GTH'}, periodicity=['XYZ']*2,ensemble='GEMC_NPT',seed=1,project_name='NH3_NPT_GEMC',restart='FALSE',pressure=1*u.bar, use_atom_name_as_symbol = False, initial_coordinate_filename = ["box1_xyz.xyz", "box2_xyz.xyz"])

In [14]:
q.mc_initialization()

scf_tolerance not specified, set as 1e-6
basis_set_filename not defined, set as BASIS_MOLOPT
potential_filename not specified, set as GTH_POTENTIALS
n_steps not specified, set as 1000
n_ff_moves not specified, set as 8
nswapmoves not specified, set as 640, will be ignore if n_box<2
output trajectory format set as XYZ
input_filename not specified, set as ['GEMC_NPT_box1.inp', 'GEMC_NPT_box2.inp']
output_filename not specified, set as NH3_NPT_GEMC_mc_output.out
temperature not defined, set as 298 K
You can change default settings in setter.mc_files


In [15]:

q.topology_filename=['hydrogen_bias.psf', 'nitrogen_bias.psf']

# move_probabilities=[pmavbmc,pmcltrans,pmhmc,pmswap,pmtraion,pmtrans,pmvolume]
#volume moves = PMVOLUME
# swap moves = PMSWAP - PMVOLUME
# AVBMC moves = PMAVBMC - PMSWAP - PMVOLUME
# an “inner” move = 1.0 - (PMAVBMC + PMSWAP + PMVOLUME)
#conformational changes = “inner” move percentage × PMTRAION
# molecular translation = “inner” move percentage × (PMTRANS - PMTRAION)
# molecular rotation = “inner” move percentage × (1.0 - PMTRANS - PMTRAION)

q.move_probabilities=[0,0.0,0,0.4,0.5,0.75,0.05]



# mol_probabilities=[[PMAVBMC_MOL,PMSWAP_MOL , PMTRAION_MOL, PMTRANS_MOL,PMROT_MOL],[]]
q.mol_probabilities=[[[1, 1],[1, 1],[1, 1],[1, 1],[1, 1]],[[1, 1],[1, 1],[1, 1],[1, 1],[1, 1]]]



#avbmc probabilities=[[AVBMC_ATOM,AVBMC_RMIN,AVBMC_RMAX,PBIAS],[]]
q.avbmc_probabilities=[[[1, 1],[1, 1],[1, 1],[1, 1]],[[1, 1],[1, 1],[1, 1],[1, 1]]]

q.charmm_potential_file='overall_charmm_bias.inp'

In [16]:
setter.mc_files(q)

MC initial structure saved as ['box1_xyz.xyz', 'box2_xyz.xyz']
This is ETA [0, 0]
MC input file saved as GEMC_NPT_box1.inp
This is ETA [0.0, 0]
MC input file saved as GEMC_NPT_box2.inp
This is ETA [0.0]


/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.8000, 0.6461, 0.4377), 0 bonds, id: 140346456148144>" is element: "H"
  warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<NN pos=( 0.4432, 0.7713, 0.0380), 0 bonds, id: 140347864960592>" is element: "N"
  warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<HH pos=( 0.2000, 0.6461, 0.4377), 0 bonds, id: 140347864957712>" is element: "H"
  warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:970: UserWarning: Guessing that "<NN pos=( 0.3432, 0.7713, 0.0380), 0 bonds, id: 140346476919824>" is element: "N"
  warn(


In [17]:
!cp2k.popt -i GEMC_NPT_box1.inp -i GEMC_NPT_box2.inp

 DBCSR| Multiplication driver                                               BLAS
 DBCSR| Multrec recursion limit                                              512
 DBCSR| Multiplication stack size                                           1000
 DBCSR| Maximum elements for images                                    UNLIMITED
 DBCSR| Multiplicative factor virtual images                                   1
 DBCSR| Multiplication size stacks                                             3
 DBCSR| Number of 3D layers                                               SINGLE
 DBCSR| Use MPI memory allocation                                              T
 DBCSR| Use RMA algorithm                                                      F
 DBCSR| Use Communication thread                                               T
 DBCSR| Communication thread load                                             87


  **** **** ******  **  PROGRAM STARTED AT               2022-10-03 12:34:05.030
 ***** ** ***  *** **   PR

In [18]:
!pwd

/mnt/d/research/software/mosdef_cp2kmcpy/mosdef_cp2kmcpy/examples/GEMC_h2_n2


In [ ]:
import ele
na = ea
le.element_from_symbol("Na")